In [1]:
from besos import eppy_funcs as ef
import sampling
from besos.parameters import GenericSelector, Parameter, RangeParameter, FieldSelector, CategoryParameter
from evaluator import EvaluatorEP, EvaluatorGeneric
from objectives import MeterReader
from problem import  Problem

In [2]:
total_range = RangeParameter(0.5, 1, name="total")
# ratio is used to decide what fraction of the total is radiant. The rest is visible
ratio_range = RangeParameter(0,1, name="ratio")

# here we create two normal selectors to simplify the function later on
lights_radiant=FieldSelector(
        class_name='Lights',
        object_name='*',
        field_name='fraction_radiant'
    )

lights_visible=FieldSelector(
    class_name='Lights',
        object_name='*',
    field_name='fraction_visible'
)

# this function sets the radiant and visible lights based on both total and ratio
def set_lights(building, total, ratio):
    lights_radiant.set(building, total*ratio)
    lights_visible.set(building, total*(1-ratio))



this parameter uses a GenericSelector to manage the function above.
Since this function takes 2 input values (as well as building) **we need to provide 2 value descriptors.**

In [3]:
parameters = [Parameter(
    selector=GenericSelector(set = set_lights),
    # the values from these descriptors are passed to the function in order
    value_descriptors=[total_range, ratio_range]
)]

In [4]:
building = ef.get_building()
problem = Problem(parameters, [MeterReader("Electricity:Facility")])
evaluator = EvaluatorEP(problem, building)

In [5]:
samples = sampling.dist_sampler(sampling.lhs, problem, 5)
evaluator.df_apply(samples, keep_input=True)

,total,ratio,Electricity:Facility
0,0.963045,0.295615,1.850526e+09
1,0.507815,0.936208,1.864769e+09
2,0.818647,0.034226,1.855776e+09
3,0.709480,0.666206,1.859549e+09
4,0.672416,0.446764,1.860741e+09


In [6]:
def only_print(first):
    """This function makes other functions which print `first` followed by the arguments they are passed"""
    def printer_func(building, *args):
        """Prints it's arguments, but does not change the building"""
        print(first, *args)
    return printer_func

a = CategoryParameter(["a", "aa", "AAA"], name="a")
b = CategoryParameter(["b", "bb", "BBB"], name="b")
c = CategoryParameter(["c", "cc", "CCC"], name="c")

parameters =[
    Parameter(GenericSelector(only_print("only a:")), a),
    Parameter(GenericSelector(only_print("c and b:")), (c , b)),
    Parameter(GenericSelector(only_print("c and a:")), (c, a)),
    Parameter(GenericSelector(only_print("----")), ()) # no inputs, will print a dashed line
]
problem = Problem(parameters, [MeterReader("Electricity:Facility")])

samples are generated for each *Descriptor* used, not for each *Parameter*

In [7]:
samples = sampling.dist_sampler(sampling.lhs, problem, 5)
samples

,a,c,b
0,AAA,c,bb
1,aa,c,bb
2,aa,CCC,b
3,a,cc,BBB
4,a,CCC,BBB


In [8]:
evaluator = EvaluatorEP(problem, building)
evaluator.df_apply(samples, keep_input=True)
# notice how the values of a,b, and c are shared when they are used by more than one funtion.
# This happens when two Parameters share a value descriptor.
# one value is generated for each value descriptor.

only a: AAA
c and b: c bb
c and a: c AAA
----
only a: aa
c and b: c bb
c and a: c aa
----
only a: aa
c and b: CCC b
c and a: CCC aa
----
only a: a
c and b: cc BBB
c and a: cc a
----
only a: a
c and b: CCC BBB
c and a: CCC a
----



,a,c,b,Electricity:Facility
0,AAA,c,bb,1.852986e+09
1,aa,c,bb,1.852986e+09
2,aa,CCC,b,1.852986e+09
3,a,cc,BBB,1.852986e+09
4,a,CCC,BBB,1.852986e+09
